In [ ]:
# Imports and stuff
import os
import time
import csv

#Libraries
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction import stop_words
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold

#Powerpuff-stuff
import helpers as HL

import cleaning as CL

# Defining paths for data: 

In [ ]:
# RUNNING ON TRAINING SET WITH CROSS VALIDATION

training_set_pos = "train_pos.txt" 
training_set_neg = "train_neg.txt"
training_set_full = "train_full.txt"
test_set = "test_data.txt"

# Creating corpus with training data:

In [ ]:
inputfiles=[training_set_pos, training_set_neg]

original_corpus, file_lengths=HL.create_corpus(inputfiles)

In [ ]:
print(original_corpus[0])

# Creating corresponding labels: 

In [ ]:
nr_pos_lines=file_lengths[0]
nr_neg_lines=file_lengths[1]

nr_lines_total=sum(file_lengths)

labels=HL.create_labels(nr_pos_lines,nr_neg_lines)

# Defining stoppingwords and removing them

In [ ]:
#Defining the list: 
#sklearn_stops = stop_words.ENGLISH_STOP_WORDS
#custom_stop_words = set(sklearn_stops)
#negation_words = set(["neither", "nor", "not", "either", "never", "no", "nobody", "nothing",
#                     "nowhere", "none", "little", "few", "rarely", "hardly", "seldom", "only",
#                     "can't", "cannot", "without", "noone"])
#tweet_words = set(["<user>","<url>"])
#custom_stop_words = custom_stop_words- negation_words
#custom_stop_words = custom_stop_words.union(tweet_words)- negation_words


custom_stop_words=set(["the","is","<url>","<user>",".",","])

In [ ]:
#removing the stopwords:
stop_words_removed_corpus=CL.remove_stopwords(original_corpus, custom_stop_words)

In [ ]:
print(stop_words_removed_corpus[0])

# Creating cluster dictionary and setting words to their cluster ID

In [ ]:
#creating dictionary
cluster_file="50mpaths2.txt"
cluster_dictionary=CL.create_dictionary(cluster_file)

In [ ]:
#setting words to their cluster ID
clusteded_corpus=CL.create_clusterized_corpus(original_corpus,cluster_dictionary) #stop_words_removed_corpus, cluster_dictionary)

In [ ]:
print(clusteded_corpus[0])

# Want to implement n-grams

In [ ]:
n_grams=2
n_grams_corpus=HL.creating_n_grams_cropus(n_grams, clusteded_corpus)


In [ ]:
print(n_grams_corpus[0])

# Starting to do shit

In [ ]:
#picking corpus
corpuses=[original_corpus,stop_words_removed_corpus,clusteded_corpus, n_grams_corpus]

In [ ]:
#Variables for measuring accuracy -----------------------

totalsvm = 0   # Accuracy measure 
totalMatSvm = np.zeros((2,2));  # Confusion matrix 

In [ ]:
for corpus in corpuses: 
    #Variables for measuring accuracy -----------------------

    totalsvm = 0   # Accuracy measure 
    totalMatSvm = np.zeros((2,2));  # Confusion matrix 

    #Running algorithm with K-fold ---------------------------

    #Creating K-fold = 10 for cross validation
    kf = StratifiedKFold(n_splits=3)

    for train_index, test_index in kf.split(corpus,labels):

        X_train = [corpus[i]for i in train_index]
        X_test  = [corpus[i]for i in test_index]
        y_train, y_test = labels[train_index], labels[test_index]

        vectorizer = TfidfVectorizer(
            min_df=10, # removing word that occure less then 10 times 
            max_df = 0.9, # remove words that are too frequent ( more then 0.8 * number of tweets )
            sublinear_tf=True, # scale the term frequency in logarithmic scale
            use_idf =True
        )
        train_corpus_tf_idf = vectorizer.fit_transform(X_train) 
        test_corpus_tf_idf = vectorizer.transform(X_test)

        model1 = LinearSVC()
        model1.fit(train_corpus_tf_idf,y_train)
        result1 = model1.predict(test_corpus_tf_idf)

        print(result1)

        totalMatSvm = totalMatSvm + confusion_matrix(y_test, result1)
        totalsvm = totalsvm+sum(y_test==result1)

    print(totalMatSvm, "\n", "Total accuracy:", totalsvm/nr_lines_total)  


In [ ]:
import gridsearch as GS

GS.gridsearch(n_grams_corpus,labels,nr_lines_total)

# Kaggle delivery:

In [ ]:
# DATA READING
start = time.time()
training_set_pos = "train_pos_full.txt" 
training_set_neg = "train_neg_full.txt"

test_set = "test_data.txt"

elapsed = time.time()

print ("Time in min:", (elapsed - start) / 60 )

In [ ]:
start = time.time()
## Defining parameters for preprocessing:
custom_stop_words=set(["the","is","<url>","<user>",".",","])

#creating dictionary
cluster_file="50mpaths2.txt"
cluster_dictionary=CL.create_dictionary(cluster_file)

#picking number of in n-grams
n_grams=2


elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

In [ ]:
start = time.time()

#### CREATING TRAINING CORPUS #### 
#original corpus:
inputfiles_train=[training_set_pos, training_set_neg]
original_corpus_train, file_lengths_train=HL.create_corpus(inputfiles_train)

#remove stopwords
#stop_words_removed_corpus_train=CL.remove_stopwords(original_corpus_train, custom_stop_words)

#using clusters
clusteded_corpus_train=CL.create_clusterized_corpus(original_corpus_train, cluster_dictionary)
#stop_words_removed_corpus_train, cluster_dictionary)

#ngrams:
n_grams_corpus_train=HL.creating_n_grams_cropus(n_grams, clusteded_corpus_train)


elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

In [ ]:
start = time.time()

#### CREATING TEST CORPUS #### 
#original corpus:
inputfiles_test=[test_set]
original_corpus_test, file_lengths_test=HL.create_corpus(inputfiles_test)

#remove stopwords
#stop_words_removed_corpus_test=CL.remove_stopwords(original_corpus_test, custom_stop_words)

#using clusters
clusteded_corpus_test=CL.create_clusterized_corpus(original_corpus_test,cluster_dictionary)#stop_words_removed_corpus_test, cluster_dictionary)

#ngrams:
n_grams_corpus_test=HL.creating_n_grams_cropus(n_grams, clusteded_corpus_test)


elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

In [ ]:
start = time.time()

##creating labels
nr_pos_lines=file_lengths_train[0]
nr_neg_lines=file_lengths_train[1]

nr_lines_total=sum(file_lengths_train)

labels=HL.create_labels(nr_pos_lines,nr_neg_lines,kaggle=True)


elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

In [ ]:
GS.gridsearch(n_grams_corpus_train,labels,nr_lines_total)

In [ ]:
start = time.time()

vectorizer = TfidfVectorizer(
    min_df=10, # removing word that occure less then 10 times 
    max_df = 0.97, # remove words that are too frequent ( more then 0.8 * number of files )
    sublinear_tf=True, # scale the term frequency in logarithmic scale
    use_idf =True, 
    #stop_words = 'english' # Removing stop-words
)

#CLASSIFYING WITH SVM
model = LinearSVC()

elapsed = time.time()

print ("Time in min:", (elapsed - start) / 60 )

In [ ]:
start = time.time()

#VECTORIZING ( Basically the preprocessing, see: TfidfVectorizer)
train_corpus_fittrans = vectorizer.fit_transform(n_grams_corpus_train) 
test_corpus_trans = vectorizer.transform(n_grams_corpus_test)

elapsed = time.time()

print ("Time in min:", (elapsed - start) / 60 )

In [ ]:
start = time.time()

# FITTING THE MODEL
model.fit(train_corpus_fittrans, labels)

elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

In [ ]:
result1 = model.predict(train_corpus_fittrans)

In [ ]:
start = time.time()

#PREDICTING
submission_prediction = model.predict(test_corpus_trans)

elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

In [ ]:
print((submission_prediction))

In [ ]:
start = time.time()

#CREATING SUBMISSION
ids = list(range(1,nr_lines_total+1))
y_pred = submission_prediction
name = "keggle_submission_cluster_27_11_new.csv"

HL.create_csv_submission(ids, y_pred, name)

elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )